In [1]:
%%time
%%sh

# The name of our algorithm
# Makoto.Sano@Mack-the-Psych.com
algorithm_name=sagemaker-vdok3-memn2n

cd container

chmod +x vdok3_sage/train
chmod +x vdok3_sage/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  37.38kB
Step 1/16 : FROM continuumio/anaconda3:4.4.0
4.4.0: Pulling from continuumio/anaconda3
8ad8b3f87b37: Pulling fs layer
fb691515f399: Pulling fs layer
6c3051db0635: Pulling fs layer
66faddd8f0d6: Pulling fs layer
66faddd8f0d6: Waiting
8ad8b3f87b37: Verifying Checksum
8ad8b3f87b37: Download complete
66faddd8f0d6: Verifying Checksum
66faddd8f0d6: Download complete
fb691515f399: Verifying Checksum
fb691515f399: Download complete
6c3051db0635: Verifying Checksum
6c3051db0635: Download complete
8ad8b3f87b37: Pull complete
fb691515f399: Pull complete
6c3051db0635: Pull complete
66faddd8f0d6: Pull complete
Digest: sha256:c6bb52bffe028b4b436b085afa4044db9b3d687a95468c92578467c9c2d4ac31
Status: Downloaded newer image for continuumio/anaconda3:4.4.0
 ---> 795ad88c47ff
Step 2/16 : RUN conda install -c anaconda setuptools
 ---> Running in 8ba630d9a3ac
Fetching package metadata ...........
Solving package specifications: .

Package plan 

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



CPU times: user 65.5 ms, sys: 16.1 ms, total: 81.7 ms
Wall time: 13min 29s


In [2]:
# S3 prefix
# Makoto.Sano@Mack-the-Psych.com
prefix = 'vdok3_memn2n_trial'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [3]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [4]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [5]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

# Makoto.Sano@Mack-the-Psych.com
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-vdok3-memn2n:latest'.format(account, region)
vdok3memn2n = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)
vdok3memn2n.fit(data_location)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-08-23 20:48:59 Starting - Starting the training job...
2020-08-23 20:49:02 Starting - Launching requested ML instances......
2020-08-23 20:50:15 Starting - Preparing the instances for training...
2020-08-23 20:50:58 Downloading - Downloading input data...
2020-08-23 20:51:10 Training - Downloading the training image......../root/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:

UnexpectedStatusException: Error for Training job sagemaker-vdok3-memn2n-2020-08-23-20-48-59-694: Failed. Reason: AlgorithmError: Exception during training: can't pickle _thread.RLock objects
Traceback (most recent call last):
  File "/opt/program/train", line 48, in train
    pickle.dump(memnd, out)
TypeError: can't pickle _thread.RLock objects


In [ ]:
from sagemaker.predictor import csv_serializer
predictor = vdok3rf.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

In [ ]:
df_in = pd.read_csv('data/Answers-Key-Stem-Passage-Aggregate_plim-Def-POST.csv')
np_in = np.vstack((np.array(df_in.columns), df_in.to_numpy()))
print(predictor.predict(np_in).decode('utf-8'))

In [ ]:
sess.delete_endpoint(predictor.endpoint)